<a href="https://colab.research.google.com/github/A7mad-Magdy77/01/blob/main/perplexity_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai plotly

In [ ]:
import numpy as np
from openai import OpenAI
import os
import plotly.graph_objects as go

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "OPENAI_API_KEY"))

def get_completion(messages, model="gpt-4o-mini", temperature=0, logprobs=True, top_logprobs=10):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        logprobs=logprobs,
        top_logprobs=top_logprobs
    )
    return response

class HumorScorer:
    def __init__(self, model="gpt-4o-mini"):
        self.model = model
        self.categories = list(range(1, 11))  # 1 to 10

    def score(self, text):
        prompt = (
            "On a scale of 1 to 10, where 1 is not funny at all and 10 is extremely funny, "
            "rate the humor level of the following text. Respond with just the number.\n\n"
            f"Text: {text}\n\nHumor Rating:"
        )
        response = get_completion([{"role": "user", "content": prompt}], model=self.model)

        logprobs = {str(cat): -float('inf') for cat in self.categories}
        for token_info in response.choices[0].logprobs.content:
            for logprob in token_info.top_logprobs:
                token = logprob.token.strip()
                if token in logprobs:
                    logprobs[token] = max(logprobs[token], logprob.logprob)

        # Convert logprobs to probabilities
        probs = {cat: np.exp(lp) for cat, lp in logprobs.items()}
        total_prob = sum(probs.values())
        normalized_probs = {cat: p / total_prob for cat, p in probs.items()}

        # Calculate weighted average score
        weighted_score = sum(int(cat) * prob for cat, prob in normalized_probs.items())

        # Normalize to 0-1 range
        final_score = (weighted_score - 1) / 9  # (score - min) / (max - min)

        print("Category probabilities:")
        for category, prob in normalized_probs.items():
            print(f"{category}: {prob:.4f}")

        return round(final_score, 3)

def plot_results(texts, scores):
    fig = go.Figure(data=[go.Bar(
        x=[f"Text {i+1}" for i in range(len(texts))],
        y=scores,
        text=[f"{score:.3f}" for score in scores],
        textposition='auto',
        hovertext=texts,
        marker_color='rgb(158,202,225)',
        marker_line_color='rgb(8,48,107)',
        marker_line_width=1.5,
        opacity=0.6
    )])

    fig.update_layout(
        title='Humor Scores for Different Texts',
        xaxis_title='Texts',
        yaxis_title='Humor Score',
        yaxis_range=[0, 1],
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Rockwell"
        )
    )

    fig.show()

# Example usage
if __name__ == "__main__":
    scorer = HumorScorer()
    texts = [
        "How do trees get on the Internet? They log in.",
        "I'm reading a book on anti-gravity. It's impossible to put down!",
        "This is a very serious statement with no humor whatsoever.",
        "The capital of France is Paris.",
        "Why don't scientists trust atoms? Because they make up everything!",
        "I told my wife she was drawing her eyebrows too high. She looked surprised.",
        "What do you call a fake noodle? An impasta!",
        "Why did the scarecrow win an award? He was outstanding in his field!",
        "I'm on a seafood diet. I see food, and I eat it.",
        "The world's population can be divided into two groups. Those who can extrapolate from incomplete data.",
        """Paddy died in a fire and was burnt pretty badly. So the morgue needed someone to identify the body. His two best friends, Seamus and Sean, were sent for. Seamus went in and the mortician pulled back the sheet.

Seamus said "Yup, he's burnt pretty bad. Roll him over".

So the mortician rolled him over. Seamus looked and said "Nope, it ain't Paddy."

The mortician thought that was rather strange and then he brought Sean in to identify the body.

Sean took a look at him and said, "Yup, he's burnt real bad, roll him over."

The mortician rolled him over and Sean looked down and said, "No, it ain't Paddy."

The mortician asked, "How can you tell?"

Sean said, "Well, Paddy had two arseholes."

"What? He had two arseholes?" asked the mortician.

"Yup, everyone knew he had two arseholes. Every time we went into town, folks would say, 'Here comes Paddy with them two arseholes...."""
    ]

    scores = []
    for text in texts:
        print(f"\nText: {text}")
        score = scorer.score(text)
        scores.append(score)
        print(f"Humor Score: {score}")
        print()

    plot_results(texts, scores)


Text: How do trees get on the Internet? They log in.
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0005
4: 0.0320
5: 0.2084
6: 0.7273
7: 0.0320
8: 0.0000
9: 0.0000
10: 0.0000
Humor Score: 0.529


Text: I'm reading a book on anti-gravity. It's impossible to put down!
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0000
5: 0.0000
6: 0.0091
7: 0.9313
8: 0.0595
9: 0.0000
10: 0.0000
Humor Score: 0.672


Text: This is a very serious statement with no humor whatsoever.
Category probabilities:
1: 0.9987
2: 0.0013
3: 0.0000
4: 0.0000
5: 0.0000
6: 0.0000
7: 0.0000
8: 0.0000
9: 0.0000
10: 0.0000
Humor Score: 0.0


Text: The capital of France is Paris.
Category probabilities:
1: 1.0000
2: 0.0000
3: 0.0000
4: 0.0000
5: 0.0000
6: 0.0000
7: 0.0000
8: 0.0000
9: 0.0000
10: 0.0000
Humor Score: 0.0


Text: Why don't scientists trust atoms? Because they make up everything!
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0000
5: 0.0004
6: 0.0754
7: 0.9180
8: 0.0062
9: 0.0000
10: 0

In [ ]:
import json
import requests
from tqdm import tqdm
import plotly.graph_objects as go
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_jokes(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad responses
    return json.loads(response.text)

def process_joke(joke, scorer):
    full_joke = f"{joke['part1']} {joke['part2']}"
    score = scorer.score(full_joke)
    return {
        'full_joke': full_joke,
        'score': score,
        'original_score': joke['score'],
        'author': joke['author']
    }

def find_funniest_jokes(jokes, scorer, top_n=10):
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_joke = {executor.submit(process_joke, joke, scorer): joke for joke in jokes}
        processed_jokes = []
        for future in tqdm(as_completed(future_to_joke), total=len(jokes), desc="Scoring jokes"):
            processed_jokes.append(future.result())

    return sorted(processed_jokes, key=lambda x: x['score'], reverse=True)[:top_n]

def plot_top_jokes(top_jokes):
    fig = go.Figure(data=[go.Bar(
        x=[f"Joke {i+1}" for i in range(len(top_jokes))],
        y=[joke['score'] for joke in top_jokes],
        text=[f"{joke['score']:.3f}" for joke in top_jokes],
        textposition='auto',
        hovertext=[joke['full_joke'] for joke in top_jokes],
        marker_color='rgb(158,202,225)',
        marker_line_color='rgb(8,48,107)',
        marker_line_width=1.5,
        opacity=0.6
    )])

    fig.update_layout(
        title='Top 10 Funniest Jokes',
        xaxis_title='Jokes',
        yaxis_title='Humor Score',
        yaxis_range=[0, 1],
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Rockwell"
        )
    )

    fig.show()


url = "https://raw.githubusercontent.com/kylecs/jokes-dataset-json/master/all.json"
scorer = HumorScorer()

print("Fetching jokes...")
jokes = fetch_jokes(url)
print(f"Fetched {len(jokes)} jokes.")

print("Finding the funniest jokes...")
top_jokes = find_funniest_jokes(jokes, scorer)

print("\nTop 10 Funniest Jokes:")
for i, joke in enumerate(top_jokes, 1):
    print(f"\n{i}. Score: {joke['score']:.3f}")
    print(f"Joke: {joke['full_joke']}")
    print(f"Author: {joke['author']}")
    print(f"Original Reddit Score: {joke['original_score']}")

plot_top_jokes(top_jokes)

Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0014
4: 0.0685
5: 0.2111
6: 0.6503
7: 0.0685
8: 0.0000
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0000
5: 0.0006
6: 0.0416
7: 0.6502
8: 0.3071
9: 0.0005
10: 0.0000
Fetching jokes...
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0016
5: 0.0515
6: 0.8062
7: 0.1401
8: 0.0006
9: 0.0000
10: 0.0000
Fetched 1000 jokes.
Finding the funniest jokes...
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0000
5: 0.0000
6: 0.0012
7: 0.6216
8: 0.3770
9: 0.0002
10: 0.0000
Category probabilities:
1: 0.0000Category probabilities:Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0002
5: 0.0022
6: 0.0919
7: 0.8719
8: 0.0338
9: 0.0000
10: 0.0000

2: 0.0000
3: 0.0019
4: 0.1061
5: 0.1061
6: 0.6106
7: 0.1749
8: 0.0003

1: 0.0000
2: 0.0000
3: 0.00009: 0.0000

4: 0.0000
10: 0.00005: 0.0000
6: 0.0008
7: 0.4373

8: 0.5615
9: 0.0004
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 

Scoring jokes:   0%|          | 0/1000 [00:00<?, ?it/s]

Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0008
4: 0.0590
5: 0.3396
6: 0.5599
7: 0.0406
8: 0.0001
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0001
4: 0.0113
5: 0.0738
6: 0.7930
7: 0.1216
8: 0.0002
9: 0.0000
10: 0.0000


Scoring jokes:   1%|          | 6/1000 [00:00<00:55, 17.82it/s]

Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0000
5: 0.0001
6: 0.0249
7: 0.9340
8: 0.0410
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0000
5: 0.0000
6: 0.0001
7: 0.1478
8: 0.8505
9: 0.0016
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0275
4: 0.7104
5: 0.2306
6: 0.0312
7: 0.0002
8: 0.0000
9: 0.00003: 0.0072
4: 0.1447
5: 0.3063
6: 0.5050
7: 0.0366
8: 0.0002
9: 0.0000
10: 0.0000

10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0015
5: 0.0271
6: 0.5451
7: 0.4245
8: 0.0017
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0001
3: 0.0809
4: 0.7674
5: 0.1333
6: 0.0180
7: 0.0003
8: 0.0000
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0004
5: 0.0077
6: 0.3722
7: 0.6136
8: 0.0060
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0019
4: 0.1061
5: 0.2246
6: 0.6105
7: 0.0568
8: 0.0001
9: 0.0000
10: 0.0000


...

Category probabilities:
1: 0.0022
2: 0.2226
3: 0.4712
4: 0.2522
5: 0.0497
6: 0.0022
7: 0.0000
8: 0.0000
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0004
5: 0.0123
6: 0.6695
7: 0.3162
8: 0.0017
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0002
4: 0.0040
5: 0.0230
6: 0.3596
7: 0.5929
8: 0.0203
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0264
3: 0.6000
4: 0.3211
5: 0.0384
6: 0.0125
7: 0.0017
8: 0.0001
9: 0.0000
10: 0.0000


Scoring jokes:  99%|█████████▉| 993/1000 [01:28<00:00,  9.96it/s]

Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0000
5: 0.0000
6: 0.0063
7: 0.7264
8: 0.2672
9: 0.0001
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0000
5: 0.0012
6: 0.1778
7: 0.7969
8: 0.0241
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0118
4: 0.3448
5: 0.1846
6: 0.3907
7: 0.0679
8: 0.0002
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0001
4: 0.0171
5: 0.1116
6: 0.8246
7: 0.0465
8: 0.0000
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0045
3: 0.2435
4: 0.5842
5: 0.1304
6: 0.0330
7: 0.0045
8: 0.0000
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0000
5: 0.0001
6: 0.0189
7: 0.8019
8: 0.1789
9: 0.0003
10: 0.0000


Scoring jokes: 100%|██████████| 1000/1000 [01:28<00:00, 11.27it/s]

Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0022
4: 0.0575
5: 0.2006
6: 0.6178
7: 0.1216
8: 0.0003
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0001
4: 0.0138
5: 0.3137
6: 0.6641
7: 0.0084
8: 0.0000
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0014
3: 0.0958
4: 0.3787
5: 0.3787
6: 0.1393
7: 0.0061
8: 0.0000
9: 0.0000
10: 0.0000
Category probabilities:
1: 0.0000
2: 0.0000
3: 0.0000
4: 0.0000
5: 0.0007
6: 0.1054
7: 0.8827
8: 0.0111
9: 0.0000
10: 0.0000

Top 10 Funniest Jokes:

1. Score: 0.802
Joke: A DEA officer stopped at our farm yesterday... "I need to inspect your farm for illegal growing of drugs."  I said, "Okay, but don't go into that field over there....."  The DEA officer verbally exploded saying, "Mister, I have the authority of the federal government with me!" Reaching into his rear pants pocket, the arrogant officer removed his badge and shoved it in my face. "See this fucking badge?! This badge means I'm allowed to go wherever I w